In [1]:
import pandas as pd
import numpy as np

# Capítulo 10 - Agregação de dados e operações em grupos

## 10.1 - Funcionamento do GroupBy

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.299370,0.576941
1,a,two,0.541959,0.585760
2,b,one,-1.700893,2.065311
3,b,two,0.818328,-1.445400
4,a,one,-0.323420,0.191684


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a   -0.026944
b   -0.441282
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -0.311395
      two     0.541959
b     one    -1.700893
      two     0.818328
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,-0.311395,0.541959
b,-1.700893,0.818328


In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005    0.541959
            2006   -1.700893
Ohio        2005    0.259479
            2006   -0.323420
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.026944,0.451462
b,-0.441282,0.309955


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.311395  0.384312
     two   0.541959  0.585760
b    one  -1.700893  2.065311
     two   0.818328 -1.445400

In [15]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterando por grupos

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.299370  0.576941
1    a  two  0.541959  0.585760
4    a  one -0.323420  0.191684
b
  key1 key2     data1     data2
2    b  one -1.700893  2.065311
3    b  two  0.818328 -1.445400


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2    data1     data2
0    a  one -0.29937  0.576941
4    a  one -0.32342  0.191684
a two
  key1 key2     data1    data2
1    a  two  0.541959  0.58576
b one
  key1 key2     data1     data2
2    b  one -1.700893  2.065311
b two
  key1 key2     data1   data2
3    b  two  0.818328 -1.4454


In [18]:
pieces = dict(list(df.groupby('key1')))

In [19]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.700893,2.065311
3,b,two,0.818328,-1.445400


In [20]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)

In [22]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.299370  0.576941
1  0.541959  0.585760
2 -1.700893  2.065311
3  0.818328 -1.445400
4 -0.323420  0.191684
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecionando uma coluna ou um subconjunto de colunas

In [23]:
df.groupby('key1')['data1']

In [24]:
df['data1'].groupby(df['key1'])

In [25]:
df.groupby('key1')[['data2']]

In [26]:
df[['data2']].groupby(df['key1'])

In [27]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.384312
     two   0.585760
b    one   2.065311
     two  -1.445400

In [28]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [29]:
s_grouped

In [30]:
s_grouped.mean()

key1  key2
a     one     0.384312
      two     0.585760
b     one     2.065311
      two    -1.445400
Name: data2, dtype: float64

### Agrupando com dicionários e Series

In [31]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [32]:
people.iloc[2:3, [1, 2]] = np.nan

In [33]:
people

,a,b,c,d,e
Joe,0.597156,0.316038,0.171463,-0.593000,0.764770
Steve,-0.239933,1.361490,2.547130,-1.497052,-2.257013
Wes,0.006706,NaN,NaN,-1.133355,0.428833
Jim,-0.066860,-0.937847,-0.467464,-0.430166,-0.121963
Travis,0.371044,0.457409,1.556960,0.556448,-0.753674


In [34]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red'}

In [35]:
by_column = people.groupby(mapping, axis=1)

In [36]:
by_column.sum()

,blue,red
Joe,-0.421537,1.677964
Steve,1.050078,-1.135456
Wes,-1.133355,0.435539
Jim,-0.897630,-1.126671
Travis,2.113408,0.074779


In [37]:
map_series = pd.Series(mapping)

In [38]:
map_series

a     red
b     red
c    blue
d    blue
e     red
dtype: object

In [39]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Agrupando com funções

In [40]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.537002,-0.621810,-0.29600,-2.156522,1.071640
5,-0.239933,1.361490,2.54713,-1.497052,-2.257013
6,0.371044,0.457409,1.55696,0.556448,-0.753674


In [41]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [42]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.006706  0.316038  0.171463 -1.133355  0.428833
  two -0.066860 -0.937847 -0.467464 -0.430166 -0.121963
5 one -0.239933  1.361490  2.547130 -1.497052 -2.257013
6 two  0.371044  0.457409  1.556960  0.556448 -0.753674

### Agrupando por níveis de índice

In [43]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

In [44]:
hier_df = pd.DataFrame(np.random.randn(4, 5),
                       columns=columns)

In [45]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.321859 -1.996561 -0.226417  0.016905 -0.891409
1     -0.313470  0.394041 -0.398129  0.227597 -0.435995
2      0.439566 -0.251454  0.298960  2.067892  1.392019
3     -0.333438 -0.729987 -0.388568  0.390141  1.201158

In [46]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 - Agregação de dados

🗒️ Métodos otimizados de groupby - pág 523

In [47]:
df

,key1,key2,data1,data2
0,a,one,-0.299370,0.576941
1,a,two,0.541959,0.585760
2,b,one,-1.700893,2.065311
3,b,two,0.818328,-1.445400
4,a,one,-0.323420,0.191684


In [48]:
grouped = df.groupby('key1')

In [49]:
grouped['data1'].quantile(0.9)

key1
a    0.373693
b    0.566406
Name: data1, dtype: float64

In [50]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [51]:
grouped.agg(peak_to_peak)

/tmp/ipykernel_3836/4193407341.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,0.865378,0.394076
b,2.519221,3.510710


In [52]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.026944  0.492831 -0.323420 -0.311395 -0.299370  0.121294   
b      2.0 -0.441282  1.781358 -1.700893 -1.071087 -0.441282  0.188523   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.541959   3.0  0.451462  0.225017  0.191684  0.384312  0.576941   
b     0.818328   2.0  0.309955  2.482447 -1.445400 -0.567722  0.309955   

                          
           75%       max  
key1                      
a     0.581350  0.585760  
b     1.187633  2.065311

### Aplicação de função nas colunas e aplicação de várias funções

In [53]:
tips = pd.read_csv('examples/tips.csv')

In [54]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [55]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [56]:
grouped_pct = tips.groupby(['day', 'smoker'])

In [57]:
grouped_pct.agg('mean')

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [58]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

/tmp/ipykernel_3836/540516406.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_pct.agg(['mean', 'std', peak_to_peak])


total_bill                               tip            \
                  mean        std peak_to_peak      mean       std   
day  smoker                                                          
Fri  No      18.420000   5.059282        10.29  2.812500  0.898494   
     Yes     16.813333   9.086388        34.42  2.714000  1.077668   
Sat  No      19.661778   8.939181        41.08  3.102889  1.642088   
     Yes     21.276667  10.069138        47.74  2.875476  1.630580   
Sun  No      20.506667   8.130189        39.40  3.167895  1.224785   
     Yes     24.120000  10.442511        38.10  3.516842  1.261151   
Thur No      17.113111   7.721728        33.68  2.673778  1.282964   
     Yes     19.190588   8.355149        32.77  3.030000  1.113491   

                              size                          tip_pct            \
            peak_to_peak      mean       std peak_to_peak      mean       std   
day  smoker                                                                     
Fri  No             2.00  2.250000  0.500000            1  0.151650  0.028123   
     Yes            3.73  2.066667  0.593617            3  0.174783  0.051293   
Sat  No             8.00  2.555556  0.784960            3  0.158048  0.039767   
     Yes            9.00  2.476190  0.862161            4  0.147906  0.061375   
Sun  No             4.99  2.929825  1.032674            4  0.160113  0.042347   
     Yes            5.00  2.578947  0.901591            3  0.187250  0.154134   
Thur No             5.45  2.488889  1.179796            5  0.160298  0.038774   
     Yes            3.00  2.352941  0.701888            2  0.163863  0.039389   

                          
            peak_to_peak  
day  smoker               
Fri  No         0.067349  
     Yes        0.159925  
Sat  No         0.235193  
     Yes        0.290095  
Sun  No         0.193226  
     Yes        0.644685  
Thur No         0.193350  
     Yes        0.151240

In [59]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

/tmp/ipykernel_3836/345979284.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])


total_bill                  tip                size            \
                   foo        bar       foo       bar       foo       bar   
day  smoker                                                                 
Fri  No      18.420000   5.059282  2.812500  0.898494  2.250000  0.500000   
     Yes     16.813333   9.086388  2.714000  1.077668  2.066667  0.593617   
Sat  No      19.661778   8.939181  3.102889  1.642088  2.555556  0.784960   
     Yes     21.276667  10.069138  2.875476  1.630580  2.476190  0.862161   
Sun  No      20.506667   8.130189  3.167895  1.224785  2.929825  1.032674   
     Yes     24.120000  10.442511  3.516842  1.261151  2.578947  0.901591   
Thur No      17.113111   7.721728  2.673778  1.282964  2.488889  1.179796   
     Yes     19.190588   8.355149  3.030000  1.113491  2.352941  0.701888   

              tip_pct            
                  foo       bar  
day  smoker                      
Fri  No      0.151650  0.028123  
     Yes     0.174783  0.051293  
Sat  No      0.158048  0.039767  
     Yes     0.147906  0.061375  
Sun  No      0.160113  0.042347  
     Yes     0.187250  0.154134  
Thur No      0.160298  0.038774  
     Yes     0.163863  0.039389

In [60]:
functions = ['count', 'mean', 'max']

In [63]:
result = grouped_pct['tip_pct', 'total_bill'].agg(functions)

/tmp/ipykernel_3836/1355840580.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped_pct['tip_pct', 'total_bill'].agg(functions)


In [64]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [65]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [66]:
ftuples = [('Durchshnitt', 'mean'), ('Abweichung', np.var)]

In [69]:
grouped_pct['tip_pct', 'total_bill'].agg(ftuples)

/tmp/ipykernel_3836/2389867931.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_pct['tip_pct', 'total_bill'].agg(ftuples)


tip_pct             total_bill            
            Durchshnitt Abweichung Durchshnitt  Abweichung
day  smoker                                               
Fri  No        0.151650   0.000791   18.420000   25.596333
     Yes       0.174783   0.002631   16.813333   82.562438
Sat  No        0.158048   0.001581   19.661778   79.908965
     Yes       0.147906   0.003767   21.276667  101.387535
Sun  No        0.160113   0.001793   20.506667   66.099980
     Yes       0.187250   0.023757   24.120000  109.046044
Thur No        0.160298   0.001503   17.113111   59.625081
     Yes       0.163863   0.001551   19.190588   69.808518

In [71]:
grouped_pct.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [72]:
grouped_pct.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Devolvendo dados agregados sem índices de linha

In [74]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 - Método apply: separar-aplicar-combinar genérico

In [83]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=False)[:n]

In [84]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [85]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [88]:
tips.groupby(['smoker', 'day']).apply(top, 
                                      n=1, 
                                      column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [89]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [90]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [91]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suprimindo as chaves de grupo

In [92]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Análise de quantis e de buckets

In [93]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [94]:
quartiles = pd.cut(frame.data1, 4)

In [95]:
quartiles[:10]

0     (-0.371, 1.367]
1      (1.367, 3.105]
2     (-0.371, 1.367]
3    (-2.108, -0.371]
4    (-2.108, -0.371]
5     (-0.371, 1.367]
6     (-0.371, 1.367]
7    (-2.108, -0.371]
8    (-2.108, -0.371]
9     (-0.371, 1.367]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.853, -2.108] < (-2.108, -0.371] < (-0.371, 1.367] < (1.367, 3.105]]

In [96]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [97]:
grouped = frame.data2.groupby(quartiles)

In [98]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.853, -2.108]",-1.470798,1.225472,19.0,-0.133703
"(-2.108, -0.371]",-2.958934,2.970584,327.0,0.039177
"(-0.371, 1.367]",-3.714736,2.868791,580.0,0.038010
"(1.367, 3.105]",-3.422776,2.153837,74.0,-0.175730


In [99]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [101]:
grouped = frame.data2.groupby(grouping)

In [102]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-1.825933,2.448563,100.0,0.030113
1,-2.429233,2.970584,100.0,0.104180
2,-2.643597,1.876030,100.0,0.072781
3,-2.958934,2.138353,100.0,-0.080710
4,-2.328608,2.543902,100.0,0.041954
5,-1.418143,2.691001,100.0,0.236681
6,-1.982256,2.471534,100.0,-0.028295
7,-2.886396,2.868791,100.0,0.005052
8,-2.144517,2.089572,100.0,-0.061337


### Exemplo: preenchendo valores ausentes com valores específicos de grupo

In [103]:
s = pd.Series(np.random.randn(6))

In [104]:
s[::2] = np.nan

In [105]:
s

0         NaN
1    1.147127
2         NaN
3    0.068057
4         NaN
5   -0.595651
dtype: float64

In [106]:
s.fillna(s.mean())

0    0.206511
1    1.147127
2    0.206511
3    0.068057
4    0.206511
5   -0.595651
dtype: float64

In [108]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [109]:
group_key = ['East'] * 4 + ['West'] * 4

In [110]:
data = pd.Series(np.random.randn(8), index=states)

In [111]:
data

Ohio          1.576925
New York      0.308869
Vermont      -0.528607
Florida      -0.253222
Oregon        0.311317
Nevada        0.470048
California    0.091450
Idaho         1.249269
dtype: float64

In [112]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [113]:
data

Ohio          1.576925
New York      0.308869
Vermont            NaN
Florida      -0.253222
Oregon        0.311317
Nevada             NaN
California    0.091450
Idaho              NaN
dtype: float64

In [114]:
data.groupby(group_key).mean()

East    0.544191
West    0.201383
dtype: float64

In [115]:
fill_mean = lambda g: g.fillna(g.mean())

In [116]:
data.groupby(group_key).apply(fill_mean)

Ohio          1.576925
New York      0.308869
Vermont       0.544191
Florida      -0.253222
Oregon        0.311317
Nevada        0.201383
California    0.091450
Idaho         0.201383
dtype: float64

In [117]:
fill_values = {'East': 0.5, 'West': -1}

In [119]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [120]:
data.groupby(group_key).apply(fill_func)

Ohio          1.576925
New York      0.308869
Vermont       0.500000
Florida      -0.253222
Oregon        0.311317
Nevada       -1.000000
California    0.091450
Idaho        -1.000000
dtype: float64

### Exemplo: amostragem aleatória e permitação

In [122]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [124]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [125]:
def draw(deck, n=5):
    return deck.sample(n)

In [126]:
draw(deck)

6D     6
AD     1
6C     6
KD    10
2H     2
dtype: int64

In [127]:
get_suit = lambda card: card[-1]

In [128]:
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   5C     5
D  4D     4
   QD    10
H  JH    10
   KH    10
S  7S     7
   2S     2
dtype: int64

In [129]:
deck.groupby(get_suit, group_keys=False).apply(draw, 2)

4C     4
6C     6
4D     4
2D     2
4H     4
KH    10
8S     8
AS     1
dtype: int64

### Exemplo: média ponderada de grupos e correlação

In [130]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

In [131]:
df

,category,data,weights
0,a,-0.590292,0.955829
1,a,-0.187309,0.898511
2,a,-1.507293,0.279704
3,a,0.058793,0.970496
4,b,0.085969,0.435988
5,b,0.168713,0.782951
6,b,1.314566,0.961516
7,b,-0.218224,0.261202


In [132]:
grouped = df.groupby('category')

In [133]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [134]:
grouped.apply(get_wavg)

category
a   -0.353371
b    0.563777
dtype: float64

In [135]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [138]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [139]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [140]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [141]:
rets = close_px.pct_change().dropna()

In [142]:
get_year = lambda x: x.year

In [143]:
by_year = rets.groupby(get_year)

In [144]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [145]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Exemplo: regressão linear nos grupos

In [147]:
import statsmodels.api as sm

In [148]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [149]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 - Tabelas pivôs e tabulação cruzada

In [150]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [151]:
tips.pivot_table(['tip_pct', 'size'], 
                 index=['time', 'day'], 
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [152]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', 
                 margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [153]:
tips.pivot_table('tip_pct', index=['time', 'smoker'],
                 columns='day',
                 aggfunc=len,
                 margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [154]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day',
                 aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

🗒️ Opções de pivot_table - pág 555

### Tabulações cruzadas: crosstab

In [163]:
us = 'USA'
jp = 'Japan'
rh = 'Right-handed'
lh = 'Left-handed'
data = pd.DataFrame({'Sample': range(1, 11),
                     'Nationality': [us, jp, us, jp, jp, jp, us, us, jp, us],
                     'Handedness': [rh, lh, rh, rh, lh, rh, rh, lh, rh, rh]})

In [164]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [166]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [167]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244